In [2]:
import cv2, os, glob
from xml.etree import ElementTree as et
import shutil

### 파일분리

In [ ]:
with open('/data/open_dt/label_frame/5detection.txt','r') as f:
    ff = f.read().split('\n')

In [ ]:
for i in range(10):
    try:
        os.mkdir(f'/data/open_dt/label_frame/{i}th_frame/')
    except:
        pass
    log = open(f"/data/open_dt/label_frame/{i}th_frame/file_list.txt",'w')
    
    for j in ff[i*1000:i*1000+1000]:
        img_name = j.split()[0]
        shutil.copy(f'/data/open_dt/label_frame/hist_img/{img_name}',f'/data/open_dt/label_frame/{i}th_frame/{img_name}')
        log.write(f"{img_name}\n")
        
    log.close()    
    

## default xml 세팅

In [ ]:
with open('./p3d712.xml','r') as f:
    parser = et.parse(f)

In [ ]:
root = parser.getroot()

In [ ]:
for i in root.findall("object"):
    root.remove(i)

In [ ]:
root.getchildren()

In [ ]:
def indent(elem, level = 0):
    i = "\n" + level + '\t'
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
            
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i
        

In [ ]:
with oprn('./default.xml','wb') as f:
    indent(root)
    er = et.tostring(root)
    f.write(er)

## xml함수

In [ ]:
def file_info_setting(path):
    with open("default.xml",'r') as f:
        root = et.parse(f).getroot()
    name = path.split("/")[-1]
    folder = path.split("/")[-2]
    root.find("filename").text = name
    root.find("path").text = path
    root.find("folder").text = folder
    img = cv2.imread(path)
    h,w,c = img.shape
    root.find('size').find("width").text = str(w)
    root.find('size').find("height").text = str(h)
    root.find('size').find("depth").text = str(c)
    
    return root, img

In [ ]:
path = "/data/open_dt/label_fram/fsgsdgsdgsdgsg.jpg"
root, img = file_info_setting(root,path)

In [ ]:
indent(root)
et.dump(root)

In [3]:
def make_object(name,bbox):
    obj = et.Element("object")
    
    el_name =et.Element("name")
    el_name.text = name
    
    pose = et.Element("pose")
    pose.text = "Unspecified"
    
    trunc = et.Element("truncated")
    trunc.text = "0"
    
    diff = et.Element("difficult")
    diff.text = "0"
    
    bndbox = et.Element("bndbox")
    
    xmin = et.Element('xmin')
    xmin.text = str(bbox[0])
    
    ymin = et.Element("ymin")
    ymin.text = str(bbox[1])
    
    xmax = et.Element("xmax")
    xmax.text = str(bbox[0] + bbox[2])
    
    ymax = et.Element("ymax")
    ymax.text = str(bbox[1] + bbox[3])
    
    bndbox.append(xmin)
    bndbox.append(ymin)
    bndbox.append(xmax)
    bndbox.append(ymax)
    
    abj.append(el_name)
    abj.append(pose)
    abj.append(trunc)
    abj.append(diff)
    abj.append(bndbox)
    
    return obj


In [ ]:
def save_xml(root, path):
    indent(root)
    with open(paht, "wb") as f:
        er = et.tostring(root)
        f.write(er)

In [ ]:
net = cv2.dnn_DetectionModel("11.train/0826/yolov4_2.cfg",'/data/kyj1103/fdsf/yolov4_2_best.weights')
net.setInputScale(1/255)
net.setInputSwapRB(True)
net.setInputSize((864,864))
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

In [ ]:
img = cv2.imread('/home/kyj1103/fjdfksjdf.jpg')
c,s,bboxes = net.detect(img,nmsThreshold=0.4, confThreshold=0.8)
obj = make_object("car",bboxes[0])

## detection

In [ ]:
net = cv2.dnn_DetectionModel("11.train/0826/yolov4_2.cfg",'/data/kyj1103/fdsf/yolov4_2_best.weights')
net.setInputScale(1/255)
net.setInputSwapRB(True)
net.setInputSize((864,864))
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

In [ ]:
file_count = 0

total_count = len(glob.glob(f"/home/kyj1103/*.jpg"))

for path in glob.glob(f"/home/kyj1103/*.jpg"): ga
    root, img = file_info_setting(path)
    c,s,bboxes = net.detect(imt, nmsThreshold = 0.4, confThreshold = 0.8)
    
    for cc, bbox in zip(c,bboxes):
        if cc[0] == 1:
            obj = make_object("license plate", bbox)
            root.append(obj)
            
    save_path = path[:-4] + ".xml"
    save_xml(root, save_path)
    file_count +=1
    print(f"{file_count} / {total_count}", end = "\r")

## log만들기

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import time

In [ ]:
def gcd(pair):
    a,b = pair
    low = min(a,b)
    for i in range(low,0,-1):
        if a%i == 0 and b%i ==0:
            return i
    assert False, 'no reach'    

In [ ]:
numbers = [(1963309,2265973),(2029292,3010101),(1231233,2344252),(12389898,12381915),(55858581,2939393),
          (1929292,2957589),(2292929,19191912),(18181818,35319398)]

In [ ]:
start = time.time()
results = list(map(gcd,numbers))
end = time.time()
total = end - start

print(f"{total:2f}s")

In [ ]:
start = time.time()
pool = ThreadPoolExecutor(max_workers = 2)
results = list(pool.map(gcd,numbers))
end = time.time()
total = end - start

print(f"{total:2f}s")